### 협업필터링

이웃구하기

In [6]:
import pandas as pd
import numpy as np
from math import sqrt


# 필요한 도구, 상수, 출력 세팅
RATING_DATA_PATH = './data/ratings.csv'  # 받아올 평점 데이터 경로 정의
np.set_printoptions(precision=2)  # 소수점 둘째 자리까지만 출력

In [7]:
def distance(user_1, user_2):
    """유클리드 거리를 계산해주는 함수"""
    return sqrt(np.sum((user_1 - user_2)**2))

In [8]:
def filter_users_without_movie(rating_data, movie_id):
    """movie_id 번째 영화를 평가하지 않은 유저들은 미리 제외해주는 함수"""
    return rating_data[~np.isnan(rating_data[:,movie_id])]
    # ~ 연산자는 비트 단위의 논리 부정(NOT) 연산자

filter_users_without_movie 함수는 파라미터로 평점 데이터 행렬과 영화 번호를 받아서 평점 데이터 행렬에서 해당 영화를 평가하지 않은 유저 정보를 미리 다 제거해 주는 함수입니다. 유저의 이웃을 구하는데 이웃들이 원하는 영화에 평점을 안 줬으면 어짜피 사용할 수 없으니까 미리 없애주기 위해 있습니다.

In [9]:
def fill_nan_with_user_mean(rating_data):
    """평점 데이터의 빈값들을 각 유저 평균 값으로 체워주는 함수"""
    filled_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    row_mean = np.nanmean(filled_data, axis=1)  # 유저 평균 평점 계산
    
    inds = np.where(np.isnan(filled_data))  # 비어 있는 인덱스들을 구한다
    filled_data[inds] = np.take(row_mean, inds[0])  #빈 인덱스를 유저 평점으로 채운다
    
    return filled_data

평점 데이터 행렬의 빈값들을 각 유저의 평균 평점으로 채워 넣어주는 함수입니다. 이 함수는 파라미터로 평점 데이터 행렬을 받고, 빈값들이 유저의 평균 평점으로 채워진 새로운 행렬을 리턴합니다.

과제: get_k_neighbors 함수

과제로는 위 함수, get_k_neighbors를 구현합니다. get_k_neighbors 함수는 파라미터로 몇 번째 유저인지를 user_id로, 평점 데이터를 rating_data로, 그리고 몇 명의 이웃들을 찾을지를 k로 받습니다. user_id는 그냥 각 행렬 안에서의 순서라고 생각하시면 됩니다. 그리고 user_id의 유저와 가장 가까운 k 명의 유저 평점 데이터를 리턴하죠.

주의하셔야 될 점은 각 행의 마지막 열은 거리 정보를 저장하는 열이기 때문에 거리 계산에서 제외해 줘야 합니다.

In [ ]:
def get_k_neighbors(user_id, rating_data, k):
    """user_id에 해당하는 유저의 이웃들을 찾아주는 함수"""
    distance_data = np.copy(rating_data)  
    # 평점 데이터를 훼손하지 않기 위해 복사
    # 마지막에 거리 데이터를 담을 열 추가한다
    distance_data = np.append(distance_data, np.zeros((distance_data.shape[0], 1)), axis=1)
    
    # 코드를 쓰세요.
    
    # 반복문을 돌면서 user_id 번째 유저가 나올 때는, 
    # 해당 데이터의 거리 정보는 무한대, np.inf로 저장해 주시면 됩니다.

    
    # 데이터를 거리 열을 기준으로 정렬한다
    distance_data = distance_data[np.argsort(distance_data[:, -1])]
    
    # 가장 가까운 k개의 행만 리턴한다 + 마지막(거리) 열은 제외한다
    return distance_data[:k, :-1]
    

# 실행 코드
# 영화 3을 본 유저들 중, 유저 0와 비슷한 유저 5명을 찾는다
rating_data = pd.read_csv(RATING_DATA_PATH, index_col='user_id').values  # 평점 데이터를 불러온다
filtered_data = filter_users_without_movie(rating_data, 3)  # 3 번째 영화를 보지 않은 유저를 데이터에서 미리 제외시킨다
filled_data = fill_nan_with_user_mean(filtered_data)  # 빈값들이 채워진 새로운 행렬을 만든다
user_0_neighbors = get_k_neighbors(0, filled_data, 5)  # 유저 0과 비슷한 5개의 유저 데이터를 찾는다
user_0_neighbors

In [11]:
def get_k_neighbors(user_id, rating_data, k):
    """user_id에 해당하는 유저의 이웃들을 찾아주는 함수"""
    distance_data = np.copy(rating_data)  
    # 평점 데이터를 훼손하지 않기 위해 복사
    # 마지막에 거리 데이터를 담을 열 추가한다
    distance_data = np.append(distance_data, np.zeros((distance_data.shape[0], 1)), axis=1)
    
    # 코드를 쓰세요.
    for i in range(len(distance_data)):
        row = distance_data[i]
        
    # 반복문을 돌면서 user_id 번째 유저가 나올 때는, 
    # 해당 데이터의 거리 정보는 무한대, np.inf로 저장해 주시면 됩니다.
        if i == user_id: # 같은 유저면 거리를 무한대로 설정 
            row[-1] = np.inf
            
        else: # 다른 유저와 마지막 열에 거리 데이터를 저장 
                row[-1] = distance(distance_data[user_id][:-1], row[:-1])
            
    # 각 행의 마지막 인덱스 : 해당 데이터가 user_id번째 user와 얼마나 
    # 가까운지 저장 >> 원하는 user 데이터 자체가 자기 자신의 이웃에 포함되지 않게
    # 스스로와의 거리는 무한대(np.inf)로 설정
    
    # 그게 아니라면, 
    # 거리를 구하는 함수 distance를 사용해서 user_id번째 유저와의 거리를 계산하고, 
    # 현재 행의 마지막 인덱스에 저장합니다.
    # 주의하셔야 될 점은 각 행의 마지막 열은 거리 정보를 저장하는 열이기 때문에 
    # 거리 계산에서 제외해 줘야 합니다
       
        
    
    # 데이터를 거리 열을 기준으로 정렬한다
    distance_data = distance_data[np.argsort(distance_data[:, -1])]
    
    # 가장 가까운 k개의 행만 리턴한다 + 마지막(거리) 열은 제외한다
    return distance_data[:k, :-1]
    

# 실행 코드
# 영화 3을 본 유저들 중, 유저 0와 비슷한 유저 5명을 찾는다
rating_data = pd.read_csv(RATING_DATA_PATH, index_col='user_id').values  # 평점 데이터를 불러온다
filtered_data = filter_users_without_movie(rating_data, 3)  # 3 번째 영화를 보지 않은 유저를 데이터에서 미리 제외시킨다
filled_data = fill_nan_with_user_mean(filtered_data)  # 빈값들이 채워진 새로운 행렬을 만든다
user_0_neighbors = get_k_neighbors(0, filled_data, 5)  # 유저 0과 비슷한 5개의 유저 데이터를 찾는다
user_0_neighbors

array([[3.18, 3.18, 3.18, 5.  , 3.18, 3.18, 2.  , 2.  , 2.  , 3.18, 3.  ,
        4.  , 2.  , 5.  , 4.  , 3.18, 3.18, 3.18, 4.  , 2.  ],
       [3.36, 5.  , 3.36, 5.  , 3.  , 3.36, 3.36, 3.  , 2.  , 4.  , 2.  ,
        3.36, 4.  , 4.  , 5.  , 4.  , 2.  , 3.36, 1.  , 3.  ],
       [2.71, 2.71, 2.  , 5.  , 2.71, 2.71, 2.71, 2.71, 2.71, 2.71, 2.71,
        2.71, 1.  , 2.71, 2.71, 2.71, 3.  , 1.  , 5.  , 2.  ],
       [2.78, 5.  , 1.  , 4.  , 2.78, 2.78, 2.78, 3.  , 1.  , 2.78, 1.  ,
        2.78, 2.78, 4.  , 2.78, 2.78, 2.  , 2.78, 2.78, 4.  ],
       [3.  , 3.  , 3.  , 5.  , 4.  , 3.  , 3.  , 4.  , 5.  , 3.  , 3.  ,
        1.  , 2.  , 1.  , 1.  , 3.  , 3.  , 3.  , 4.  , 3.  ]])

movie_id가 3인 유저를 제외하는 이유는, 해당 영화를 평가하지 않은 유저들은 user_id 0인 유저의 movie_id 3인 영화에 대한 평점을 예측하는 데 사용될 수 없기 때문입니다.